In [1]:
import numpy as np
import pandas as pd

## Load Data

In [2]:
# load data
var_list = ['id',
            'age',
            'sex',
            'educ',
            'marital',
            'childs',
            'wrkstat',
            'income',
            'incom16', # family income while growing up
            'partyid',
            'relig',
            'fund', # fundamentalism/ liberalism of respondents religion
            'reliten',
            'courts',
            'cappun'
] # List of variables you want to save
#
output_filename = 'selected_gss_data.csv' # Name of the file you want to save the data to
#
modes = ['w','a'] # Has write mode and append mode
phase = 0 # Starts in write mode; after one iteration of loop, switches to append mode
#
col_names = []
#
for k in range(3): # for each chunk of the data
    url = 'https://github.com/DS3001/project_gss/raw/main/gss_chunk_' + str(1+k) + '.parquet' # Create url to the chunk to be processed
    print(url) # Check the url is correct
    df = pd.read_parquet(url) # Download this chunk of data
    # print(df.head()) # Visually inspect the first few rows
    global col_names
    col_names = df.columns
    df.loc[:,var_list].to_csv(output_filename, # specifies target file to save the chunk to
                              mode=modes[phase], # control write versus append
                              header=var_list, # variable names
                              index=False) # no row index saved
    phase = 1 # Switch from write mode to append mode

https://github.com/DS3001/project_gss/raw/main/gss_chunk_1.parquet
https://github.com/DS3001/project_gss/raw/main/gss_chunk_2.parquet
https://github.com/DS3001/project_gss/raw/main/gss_chunk_3.parquet


In [3]:
# df = pd.read_csv("/content/selected_gss_data.csv", low_memory=False) # colab line
df = pd.read_csv("selected_gss_data.csv", low_memory=False) # rivanna line

## Intial Exploration

In [113]:
df.head()

,id,age,sex,educ,marital,childs,wrkstat,income,incom16,partyid,relig,fund,reliten,courts,cappun
0,1,23.0,female,16.0,never married,0.0,working full time,NaN,average,"independent, close to democrat",jewish,liberal,NaN,about right,NaN
1,2,70.0,male,10.0,married,5.0,retired,NaN,above average,not very strong democrat,catholic,moderate,NaN,not harshly enough,NaN
2,3,48.0,female,12.0,married,4.0,working part time,NaN,average,"independent (neither, no response)",protestant,moderate,NaN,not harshly enough,NaN
3,4,27.0,female,17.0,married,0.0,working full time,NaN,average,not very strong democrat,other,NaN,NaN,about right,NaN
4,5,61.0,female,12.0,married,2.0,keeping house,NaN,below average,strong democrat,protestant,moderate,NaN,not harshly enough,NaN


In [5]:
df.dtypes

id         object
age        object
sex        object
educ       object
marital    object
childs     object
wrkstat    object
income     object
incom16    object
partyid    object
relig      object
fund       object
reliten    object
courts     object
cappun     object
dtype: object

The code below is to see if there are any exact duplicate rows, it turns out there are 2 that are just the column headers, probably from the way we imported the 3 chunks of data. I will drop both of these. 

In [4]:
# find duplicate rows
dup_mask = df.duplicated(keep=False)
dup_rows = df[dup_mask]

dup_rows

,id,age,sex,educ,marital,childs,wrkstat,income,incom16,partyid,relig,fund,reliten,courts,cappun
24130,id,age,sex,educ,marital,childs,wrkstat,income,incom16,partyid,relig,fund,reliten,courts,cappun
48261,id,age,sex,educ,marital,childs,wrkstat,income,incom16,partyid,relig,fund,reliten,courts,cappun


In [5]:
# drop duplicate rows 
df = df.drop_duplicates(keep=False)

# reset indices for future use
df = df.reset_index(drop=True)

In [6]:
df.shape

(72390, 15)

In [10]:
for each in df.columns:
    print(f"{each}: {df[each].isnull().sum()}")

id: 0
age: 769
sex: 112
educ: 263
marital: 51
childs: 261
wrkstat: 36
income: 8951
incom16: 13741
partyid: 485
relig: 437
fund: 5333
reliten: 11304
courts: 16279
cappun: 11543


## Clean CAPPUN Variable 

Choices made: encode "nan" as new category "no response"

In [11]:
df['cappun'].value_counts()

cappun
favor     42181
oppose    18666
Name: count, dtype: int64

In [7]:
df['cappun'] = df['cappun'].replace(np.nan, 'no response')

In [8]:
df['cappun'].value_counts()

cappun
favor          42181
oppose         18666
no response    11543
Name: count, dtype: int64

## Clean COURTS variable

Choices made: encode "nan" as new category "no response"

In [14]:
df['courts'].value_counts()

courts
not harshly enough    43636
about right            7906
too harshly            4569
Name: count, dtype: int64

In [9]:
df['courts'] = df['courts'].replace(np.nan, 'no response')

In [48]:
df['courts'].value_counts()

courts
not harshly enough    43636
no response           16279
about right            7906
too harshly            4569
Name: count, dtype: int64

## Clean INCOME variables 

income = total family income last year before taxes <br>
incom16 = family income while growing up 

Choices made: for incom16, encode "nan" as new category "no response"

In [17]:
df['incom16'].value_counts()

incom16
average                 28368
below average           14732
above average            9069
far below average        5334
far above average        1136
lived in institution       10
Name: count, dtype: int64

In [10]:
df['incom16'] = df['incom16'].replace(np.nan, 'no response')

In [50]:
df['incom16'].value_counts()

incom16
average                 28368
below average           14732
no response             13741
above average            9069
far below average        5334
far above average        1136
lived in institution       10
Name: count, dtype: int64

In [20]:
df['income'].value_counts()

income
$25,000 or more       34785
$10,000 to $14,999     6850
$20,000 to $24,999     5528
$15,000 to $19,999     5301
$8,000 to $9,999       2285
$1,000 to $2,999       1412
$7,000 to $7,999       1315
$5,000 to $5,999       1314
$3,000 to $3,999       1309
$6,000 to $6,999       1249
$4,000 to $4,999       1189
under $1,000            902
Name: count, dtype: int64

In [11]:
df['income'] = df['income'].replace(np.nan, 'no response')

In [52]:
df['income'].value_counts()

income
$25,000 or more       34785
no response            8951
$10,000 to $14,999     6850
$20,000 to $24,999     5528
$15,000 to $19,999     5301
$8,000 to $9,999       2285
$1,000 to $2,999       1412
$7,000 to $7,999       1315
$5,000 to $5,999       1314
$3,000 to $3,999       1309
$6,000 to $6,999       1249
$4,000 to $4,999       1189
under $1,000            902
Name: count, dtype: int64

## Cleaning Religion Variables

relig = religious preference <br>
fund =  fundamentalism/ liberalism of respondents religion <br>
reliten = how strong is their religious preference

Choices made:
- set relig = other if fund or reliten have specific values 
- set relig = none if fund or reliten say no religion
- set fund = no religion if relig = none 
- set reliten = no religion if relig = none 
-  take out the (vol.) from somewhat strong in reliten
- convert all other nan to no response 

In [23]:
df['relig'].value_counts()

relig
protestant                 40125
catholic                   17242
none                        9895
jewish                      1421
other                       1224
christian                   1000
buddhism                     272
muslim/islam                 201
orthodox-christian           176
inter-nondenominational      159
hinduism                     158
other eastern religions       44
native american               36
Name: count, dtype: int64

In [12]:
relig_mask = df['relig'].isnull()
relig_null = df.loc[relig_mask, ['relig','fund','reliten']]
relig_null['fund'].value_counts() # two obs with relig = nan have fund = liberal
relig_null['reliten'].value_counts() # some obs with relig = nan have reliten = no religion, strong, or not very strong 

reliten
no religion        12
strong              5
not very strong     5
Name: count, dtype: int64

#### impute some values of relig based on fund or reliten values 

In [13]:
# set relig = other if fund = liberal
relig_null.loc[df['fund']=='liberal',:] # see they currently have relig = NaN 
mask = (df['relig'].isnull()) & (df['fund']=='liberal') # conditions for row selection
df.loc[mask,'relig'] = 'other' # set relig value to other 
df.iloc[[70309,70614],:] # check work based on index values

,id,age,sex,educ,marital,childs,wrkstat,income,incom16,partyid,relig,fund,reliten,courts,cappun
70309,1464,27.0,female,12.0,never married,2.0,working part time,"$15,000 to $19,999",below average,not very strong democrat,other,liberal,NaN,about right,no response
70614,1769,43.0,male,12.0,never married,5.0,working full time,"$25,000 or more",below average,other party,other,liberal,NaN,too harshly,favor


In [14]:
# set relig = none if reliten = no religion
relig_null.loc[df['reliten']=='no religion',:] # see they currently have relig = NaN 
mask = (df['relig'].isnull()) & (df['reliten']=='no religion') # conditions for row selection
df.loc[mask,'relig'] = 'none' # set relig value to other 
df['relig'].value_counts() # check work based on increased 'none' value counts

relig
protestant                 40125
catholic                   17242
none                        9907
jewish                      1421
other                       1226
christian                   1000
buddhism                     272
muslim/islam                 201
orthodox-christian           176
inter-nondenominational      159
hinduism                     158
other eastern religions       44
native american               36
Name: count, dtype: int64

In [15]:
# set relig = other if reliten = strong or not very strong 
relig_null.loc[(df['reliten']=='strong') | (df['reliten']=='not very strong'),:] # see they currently have relig = NaN 
mask = (df['relig'].isnull()) & ((df['reliten']=='strong') | (df['reliten']=='not very strong')) # conditions for row selection
df.loc[mask,'relig'] = 'other' # set relig value to other 
df['relig'].value_counts() # check work based on increased 'other' value counts

relig
protestant                 40125
catholic                   17242
none                        9907
jewish                      1421
other                       1236
christian                   1000
buddhism                     272
muslim/islam                 201
orthodox-christian           176
inter-nondenominational      159
hinduism                     158
other eastern religions       44
native american               36
Name: count, dtype: int64

In [28]:
# set all other nan relig values to 'no response'
df['relig'] = df['relig'].replace(np.nan, 'no response')

### impute some values of fund based on relig 

In [141]:
df['fund'].value_counts()

fund
moderate          28283
fundamentalist    19798
no religion        9907
liberal            9705
no response        4697
Name: count, dtype: int64

In [16]:
# set fund = no religion if relig = none 
df.loc[(df['relig']=='none'),'fund'] = 'no religion'

In [17]:
# set all other nan fund values to 'no response'
df['fund'] = df['fund'].replace(np.nan, 'no response')

### reliten

In [26]:
df['reliten'].value_counts()

reliten
not very strong    24297
strong             23252
no religion         9907
no response         9027
somewhat strong     5907
Name: count, dtype: int64

In [19]:
# set reliten = no religion if relig = none 
df.loc[(df['relig']=='none'),'reliten'] = 'no religion'

In [21]:
# take out the (vol.) from somewhat strong
df['reliten'] = df['reliten'].replace('somewhat strong (vol.)', 'somewhat strong')

In [24]:
# set all other nan reliten values to 'no response'
df['reliten'] = df['reliten'].replace(np.nan, 'no response')

## Clean political party variable

Choices made: since 'no reponse' is already lumped into the 'independent (neither, no response)' category and we cannot separate the no responses from the independents, I am coding all the nan values to be 'independent (neither, no response)' so I don't create a separate extra 'no response' category. I also reformatted the text to say 'independent, neither, or no response' to make it clearer, though this variable could have been recorded better to begin with.

In [33]:
df['partyid'].value_counts()

partyid
not very strong democrat                14286
independent, neither, or no response    12025
strong democrat                         11795
not very strong republican              10678
independent, close to democrat           8663
strong republican                        7273
independent, close to republican         6378
other party                              1292
Name: count, dtype: int64

In [32]:
# set all nan values to 'independent (neither, no response)'
df['partyid'] = df['partyid'].replace(np.nan, 'independent (neither, no response)')

# reformat
df['partyid'] = df['partyid'].replace('independent (neither, no response)', 'independent, neither, or no response')

## Clean WRKSTAT variable (occupation)

Choices made: since there are only 36 nan values, I lumped them in with 'other' instead of creating a separate 'no response' category 

In [38]:
df['wrkstat'].value_counts()

wrkstat
working full time                                                             35267
retired                                                                       10886
keeping house                                                                 10764
working part time                                                              7430
unemployed, laid off, looking for work                                         2621
in school                                                                      2187
other                                                                          1679
with a job, but not at work because of temporary illness, vacation, strike     1556
Name: count, dtype: int64

In [37]:
# set all nan values to 'other'
df['wrkstat'] = df['wrkstat'].replace(np.nan, 'other')

## Clean CHILDS variable (number of children)

Choices made: coerce to numeric, leave nan rows as they are. This is so we don't lose other data if this variable is not heavily used in the analysis, but the nan values can be easily filtered out if it is (use df.loc[df['childs'].notnull(),:])

In [43]:
df['childs'] = pd.to_numeric(df['childs'], errors='coerce')

## Clean MARITAL variable

Choices made: convert all nan values (there are only 51) to 'no response'

In [50]:
df['marital'].value_counts()

marital
married          37590
never married    15910
divorced          9642
widowed           6756
separated         2441
no response         51
Name: count, dtype: int64

In [49]:
# set all nan values to 'no response'
df['marital'] = df['marital'].replace(np.nan, 'no response')

## Clean EDUC variable

Choices made: coerce to numeric, then create new column 'educ_level' converting numeric values to education level categories based on the mapping from the codebook, encoding nan values as 'no reponse' and leaving original educ variable in the dataframe for reference. 

In [66]:
# convert educ to numeric
df['educ'] = pd.to_numeric(df['educ'], errors='coerce')

In [67]:
mapping = {0: 'no formal schooling',
           1: '1st grade',
           2: '2nd grade',
           3: '3rd grade',
           4: '4th grade',
           5: '5th grade',
           6: '6th grade',
           7: '7th grade',
           8: '8th grade',
           9: '9th grade',
           10: '10th grade',
           11: '11th grade',
           12: '12th grade',
           13: '1 year of college',
           14: '2 years of college',
           15: '3 years of college',
           16: '4 years of college',
           17: '5 years of college',
           18: '6 years of college',
           19: '7 years of college',
           20: '8 years of college',
          np.nan: 'no response'}
df['educ_level'] = df['educ'].replace(mapping) # create new column with mapping

df.loc[:,['educ','educ_level']].head() # check work

,educ,educ_level
0,16.0,4 years of college
1,10.0,10th grade
2,12.0,12th grade
3,17.0,5 years of college
4,12.0,12th grade


## Clean SEX variable

Choices made: convert all nan values (there are only 112) to 'no response'

In [58]:
df['sex'].value_counts()

sex
female         40301
male           31977
no response      112
Name: count, dtype: int64

In [57]:
# set all nan values to 'no response'
df['sex'] = df['sex'].replace(np.nan, 'no response')

## Clean AGE variable

Choices made: coerce to numeric, leave nan rows as they are. Similar to CHILDS, this is so we don't lose other data if this variable is not heavily used in the analysis, but the nan values can be easily filtered out if it is (use df.loc[df['age'].notnull(),:])

In [59]:
df['age'] = pd.to_numeric(df['age'], errors='coerce')

## Final Product!

In [68]:
# put educ level variable next to educ for easy understanding 
col_order = ['id', 'age', 'sex', 'educ', 'educ_level', 'marital', 'childs', 'wrkstat', 'income',
       'incom16', 'partyid', 'relig', 'fund', 'reliten', 'courts', 'cappun'
       ]

df = df.loc[:,col_order]

In [69]:
df.head()

,id,age,sex,educ,educ_level,marital,childs,wrkstat,income,incom16,partyid,relig,fund,reliten,courts,cappun
0,1,23.0,female,16.0,4 years of college,never married,0.0,working full time,no response,average,"independent, close to democrat",jewish,liberal,no response,about right,no response
1,2,70.0,male,10.0,10th grade,married,5.0,retired,no response,above average,not very strong democrat,catholic,moderate,no response,not harshly enough,no response
2,3,48.0,female,12.0,12th grade,married,4.0,working part time,no response,average,"independent, neither, or no response",protestant,moderate,no response,not harshly enough,no response
3,4,27.0,female,17.0,5 years of college,married,0.0,working full time,no response,average,not very strong democrat,other,no response,no response,about right,no response
4,5,61.0,female,12.0,12th grade,married,2.0,keeping house,no response,below average,strong democrat,protestant,moderate,no response,not harshly enough,no response


In [70]:
for each in df.columns:
    print(f"{each}: {df[each].isnull().sum()}")

id: 0
age: 769
sex: 0
educ: 263
educ_level: 0
marital: 0
childs: 261
wrkstat: 0
income: 0
incom16: 0
partyid: 0
relig: 0
fund: 0
reliten: 0
courts: 0
cappun: 0


In [71]:
df.dtypes

id             object
age           float64
sex            object
educ          float64
educ_level     object
marital        object
childs        float64
wrkstat        object
income         object
incom16        object
partyid        object
relig          object
fund           object
reliten        object
courts         object
cappun         object
dtype: object

In [73]:
 df.to_csv('clean_gss_data.csv', index=False) 